In [82]:
import psycopg2
import psycopg2.extras as extras

import pandas as pd
from dotenv import dotenv_values

# Get the next month 
* Set Window Sheduler to monthly run at the last day of The month
* No matter what day you run this job , the program will  get PMs of the  next month

In [83]:
from datetime import datetime,date
from dateutil import relativedelta

#dt = datetime.strptime(datetime(2023,12,4,10,0).strftime('%Y-%m-%d'),'%Y-%m-%d')
dt = datetime.strptime(datetime.now().strftime('%Y-%m-%d'),'%Y-%m-%d')
print(dt)

first_day_next_month = dt + relativedelta.relativedelta(months=1, day=1)
first_day_2next_month = dt + relativedelta.relativedelta(months=2, day=1)
print(first_day_next_month )   
print(first_day_2next_month ) 


2023-11-28 00:00:00
2023-12-01 00:00:00
2024-01-01 00:00:00


In [84]:
env_path=r'D:\PythonDev\Yit\SMartApp_ScriptDev\.env'
config = dotenv_values(dotenv_path=env_path)

In [85]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [86]:
# https://github.com/technqvi/SMartApp/blob/main/app/pm_doc_manager/pm_export.py

sql_pm=f"""
    select ac.company_full_name as "ชื่อลูกค้า",
    ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" ,
    ap.project_name as "ชื่อโครงการ",
    TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",
    TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
    pm.remark as  "งวดPM",
    ae.employee_name as "หัวหน้าทีม",
    (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Engineer"
     ,ac.is_managed_by_admin 
    from app_preventivemaintenance pm
    left join app_project ap on ap.id = pm.project_id
    left join app_company ac on ac.id = ap.company_id
    left join app_employee ae on ae.id =pm.team_lead_id
    where  pm.planned_date>='{first_day_next_month}' and pm.planned_date<'{first_day_2next_month}'
    order by  ac.company_full_name,ap.enq_id,pm.remark
    """
print(sql_pm)


    select ac.company_full_name as "ชื่อลูกค้า",
    ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" ,
    ap.project_name as "ชื่อโครงการ",
    TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",
    TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
    pm.remark as  "งวดPM",
    ae.employee_name as "หัวหน้าทีม",
    (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Engineer"
     ,ac.is_managed_by_admin 
    from app_preventivemaintenance pm
    left join app_project ap on ap.id = pm.project_id
    left join app_company ac on ac.id = ap.company_id
    left join app_employee ae on ae.id =pm.team_lead_id
    where  pm.planned_date>='2023-12-01 00:00:00' and pm.planned_date<'2024-01-01 00:00:00'
    order by  ac.company_full_name,ap.enq_id,pm.remark
    


In [87]:
dfPM_All=list_data(sql_pm,None,get_postgres_conn())
dfPM_All.info()

dfPM_SM=dfPM_All.query("is_managed_by_admin==False")
dfPM_SM=dfPM_SM.drop(columns=['is_managed_by_admin'])

dfPM_NoneSM=dfPM_All.query("is_managed_by_admin==True")
dfPM_NoneSM=dfPM_NoneSM.drop(columns=['is_managed_by_admin'])

dfPM_All=None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ชื่อลูกค้า           181 non-null    object
 1   เลขที่สัญญา          3 non-null      object
 2   ENQ                  181 non-null    object
 3   ชื่อโครงการ          181 non-null    object
 4   แผนจะทำPM            181 non-null    object
 5   วันสุดท้ายที่ทำPM    181 non-null    object
 6   งวดPM                181 non-null    object
 7   หัวหน้าทีม           181 non-null    object
 8   Engineer             166 non-null    object
 9   is_managed_by_admin  181 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 13.0+ KB


In [88]:
dfPM_SM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 2 to 180
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ชื่อลูกค้า         114 non-null    object
 1   เลขที่สัญญา        0 non-null      object
 2   ENQ                114 non-null    object
 3   ชื่อโครงการ        114 non-null    object
 4   แผนจะทำPM          114 non-null    object
 5   วันสุดท้ายที่ทำPM  114 non-null    object
 6   งวดPM              114 non-null    object
 7   หัวหน้าทีม         114 non-null    object
 8   Engineer           99 non-null     object
dtypes: object(9)
memory usage: 8.9+ KB


In [89]:
dfPM_NoneSM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 0 to 176
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ชื่อลูกค้า         67 non-null     object
 1   เลขที่สัญญา        3 non-null      object
 2   ENQ                67 non-null     object
 3   ชื่อโครงการ        67 non-null     object
 4   แผนจะทำPM          67 non-null     object
 5   วันสุดท้ายที่ทำPM  67 non-null     object
 6   งวดPM              67 non-null     object
 7   หัวหน้าทีม         67 non-null     object
 8   Engineer           67 non-null     object
dtypes: object(9)
memory usage: 5.2+ KB
